# Energy Loss Scoring


## Import Libraries

In [1]:
import os
import requests
import json
import ast
import pandas as pd
import yaml

## Retrieve the Credential

In [2]:
credentials_file = '../../config/credentials/wml-credentials.json'

with open(credentials_file, "r") as f:
    credentials = json.load(f)

In [3]:
# Assuming you have these variables set somewhere in your script
USERNAME = credentials["username"]
API_KEY = credentials["apikey"]
DATAPLATFORM_URL = credentials["url"]

In [4]:
headers = {
    'Content-type': 'application/json'
}
data = {
    'username': USERNAME,
    'api_key': API_KEY
}

In [5]:
response = requests.post(
    f"{DATAPLATFORM_URL}/icp4d-api/v1/authorize", 
    headers=headers, 
    data=json.dumps(data),
    verify=False  # This corresponds to the '-k' or '--insecure' flag in curl
)
token = response.text
dict_token = ast.literal_eval(token)
bearer_token = dict_token['token']

/Users/jzhou/anaconda3/envs/mf/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## Load Dataset

In [6]:
df = pd.read_csv('../data/training.csv')

In [7]:
df["Timestamp"] = df["Timestamp"].astype(str)

#### Pick up only few lines of the rows

In [58]:
df = df.tail(200)

In [59]:
df.columns

Index(['Timestamp', 'Active Power (KW)'], dtype='object')

In [60]:
df_power = df[['Timestamp', 'Active Power (KW)']]

In [61]:
df_power = df_power.reset_index(drop=True)

## Send the Data for Scoring

In [48]:
model_factory_config_file_name = "../config/deployment_info_three_kpis_loss_rate.yml"

with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

{'cp4d_url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-16', 'cp4d_urls': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-15', 'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'}, 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions'], 'state': 'ready'}


In [49]:
# retrieve the CP4D endpoint at the IBM Watson Studio
scoring_url = model_factory_config['cp4d_url']
scoring_url

'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-16'

In [71]:
# Select the relevant columns
array_of_input_fields = ['Timestamp', 'Active Power (KW)']
array_of_values_to_be_scored = df_power[array_of_input_fields].values.tolist()

print("Input Data")
print(array_of_input_fields)
df[array_of_input_fields].head(10)

Input Data
['Timestamp', 'Active Power (KW)']


,Timestamp,Active Power (KW)
19148,2023-04-24 23:30:00,39.194
19149,2023-04-24 23:45:00,38.680
19150,2023-04-25 00:00:00,38.819
19151,2023-04-25 00:45:00,34.548
19152,2023-04-25 01:00:00,38.182
19153,2023-04-25 01:15:00,35.722
19154,2023-04-25 01:30:00,36.709
19155,2023-04-25 01:45:00,34.710
19156,2023-04-25 02:00:00,34.413
19157,2023-04-25 02:15:00,31.407


In [51]:
array_of_values_to_be_scored = df_power[array_of_input_fields].values.tolist()

In [52]:
%%time
import urllib3, requests, json

# we can have this from 
#scoring_url = 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/60a32c38-e993-4f87-91c2-519b3df7b902/predictions?version=2023-11-09'
#scoring_url = 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-16'

# NOTE: you must construct mltoken based on provided documentation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + bearer_token}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": array_of_input_fields, "values": array_of_values_to_be_scored}]}
print("Input payload")
print(json.dumps(payload_scoring, indent=2))

response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header, verify=False)
print("Scoring response")
predictions = json.loads(response_scoring.text)
print(json.dumps(predictions, indent=2))

Input payload
{
  "input_data": [
    {
      "fields": [
        "Timestamp",
        "Active Power (KW)"
      ],
      "values": [
        [
          "2023-04-24 23:30:00",
          39.194
        ],
        [
          "2023-04-24 23:45:00",
          38.68
        ],
        [
          "2023-04-25 00:00:00",
          38.819
        ],
        [
          "2023-04-25 00:45:00",
          34.548
        ],
        [
          "2023-04-25 01:00:00",
          38.182
        ],
        [
          "2023-04-25 01:15:00",
          35.722
        ],
        [
          "2023-04-25 01:30:00",
          36.709
        ],
        [
          "2023-04-25 01:45:00",
          34.71
        ],
        [
          "2023-04-25 02:00:00",
          34.413
        ],
        [
          "2023-04-25 02:15:00",
          31.407
        ],
        [
          "2023-04-25 02:30:00",
          30.24
        ],
        [
          "2023-04-25 02:45:00",
          32.652
        ],
        [
       

/Users/jzhou/anaconda3/envs/mf/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Scoring response
{
  "trace": "532aae2c43200c726fd1706685e5691d",
  "errors": [
    {
      "code": "score_processing_failure",
      "message": "window must be an integer 0 or greater"
    }
  ],
  "status_code": 400
}
CPU times: user 49.1 ms, sys: 8.81 ms, total: 57.9 ms
Wall time: 1.09 s


In [35]:
output = predictions["predictions"][0]["values"]

In [48]:
output

[[[0.37524174246050745, 0.15349099707584696, 0.5372184897654644]],
 [[0.3727812350406893, 0.1515724009108878, 0.5305034031881073]],
 [[0.37240465932663164, 0.15127977944797094, 0.5294792280678983]],
 [[0.3512063188128291, 0.1352431127104923, 0.4733508944867231]],
 [[0.3481570129248444, 0.13300687266463126, 0.4655240543262094]],
 [[0.34610499441519066, 0.13151201757130757, 0.46029206149957647]],
 [[0.34744135950215416, 0.1324846180542437, 0.4636961631898529]],
 [[0.34782910483484925, 0.13276745709578905, 0.46468609983526166]],
 [[0.3388080421254189, 0.12626165985438836, 0.44191580949035925]],
 [[0.32848013403542364, 0.11900505704776203, 0.4165176996671671]],
 [[0.3129966491144088, 0.10851052972886907, 0.37978685405104173]],
 [[0.3080708472953566, 0.1052688680444636, 0.3684410381556226]],
 [[0.30269746290090954, 0.1017861737383334, 0.3562516080841669]],
 [[0.2997438966012446, 0.09989567063073075, 0.3496348472075576]],
 [[0.30088798468166583, 0.100625966450823, 0.3521908825778805]],
 [[0.

## Combine the Results

In [63]:
reshaped_data = [row[0] for row in output]

In [64]:
column_names = ['load_factor','loss_factor','energyloss']

In [65]:
predictions_df = pd.DataFrame(reshaped_data, columns=column_names)

In [66]:
predictions_df = predictions_df.astype(int)

In [67]:
display(predictions_df)

,load_factor,loss_factor,energyloss
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
95,0,0,0
96,0,0,0
97,0,0,0
98,0,0,0


In [69]:
df_merged = pd.concat([df_power, predictions_df], axis=1)

In [70]:
display(df_merged.tail(10))

,Timestamp,Active Power (KW),load_factor,loss_factor,energyloss
90,2023-04-25 22:15:00,51.263,0,0,0
91,2023-04-25 22:30:00,52.740,0,0,0
92,2023-04-25 22:45:00,57.983,0,0,0
93,2023-04-25 23:00:00,57.136,0,0,1
94,2023-04-25 23:15:00,49.598,0,0,1
95,2023-04-25 23:30:00,50.980,0,0,0
96,2023-04-25 23:45:00,49.668,0,0,0
97,2023-04-26 00:00:00,45.915,0,0,0
98,2023-04-26 00:15:00,47.550,0,0,0
99,2023-04-26 00:30:00,44.732,0,0,0
